In [732]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

In [733]:
# DATA IMPORT

# Traffic map data import
data = pd.read_csv('U:\moliver\In Progress\Airports\GPS Volume.csv') 

# Line chart data import
line_df = pd.read_csv('U:\moliver\In Progress\Airports\Line Chart Animation\line_df_input.csv')


# Limit map data to exclude 2021 (already excluded in line chart data)
df = data[(data['WEEK']<=104)]

# Limit df to top 100 airports based on 2019 + 2020 GPS Enplanements
top100 = df[['STORE_ID','GPS ENPLANEMENTS']] \
    .groupby(['STORE_ID']) \
    .sum() \
    .sort_values(['GPS ENPLANEMENTS'], ascending=False) \
    .head(100)

df = df.join(top100,on='STORE_ID',how='inner',rsuffix='_R').drop(['GPS ENPLANEMENTS_R'],axis=1).reset_index(drop=True)


In [734]:
# Create Dataframe to convert month from numeric to text

MONTH_DESC = pd.DataFrame(
    data = ['January','February','March','April','May','June','July','August','September','October','November','December'],
    dtype = 'category',
    index = [1,2,3,4,5,6,7,8,9,10,11,12],
    columns=['MONTH_DESC'])

# Iterable key dataframe for updating annotations in animaiton

master_lst = df[['WEEK','YEAR','MONTH','TRUE MONTHLY ENPLANED']] \
    .groupby(['WEEK']).first().reset_index(drop = False) \
    .join(MONTH_DESC,on='MONTH',how='outer')


In [735]:
# TRAFFIC MAP

# Create Map Figure
fig = px.scatter_geo(
    df,
    lat=df.LATITUDE,
    lon=df.LONGITUDE,
    hover_name=df.STORE_ID,
    size='GPS ENPLANEMENTS',
    color='GPS ENPLANEMENTS',
    range_color=[100000,500000],
    width=1800,
    height=680,
    animation_frame=df.WEEK,
    animation_group=df.STORE_ID)

# Formatting Updates
fig.update_layout(
    title = 'Estimated Traffic from Top 100 Airports Jan 2019-Dec 2020 <i>(Refresh to Replay)</i>',
    geo_scope = 'usa',
    font_color = '#1c0a38'
    )

fig.layout.title.font.size = 18
fig.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 250
fig.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 200

fig.layout.updatemenus[0].buttons[0].visible = False
fig.layout.updatemenus[0].buttons[1].visible = False
fig.layout.coloraxis.colorbar.title.text = 'Estimated Weekly Enplanements'
fig.layout.sliders[0].visible = False

# for loop to add annotations to each animation frame created by px.scatter_geo
for i, frame in enumerate(fig.frames):
     fig.frames[i].layout["annotations"] = [
        go.layout.Annotation(
            x=.0,
            y=.0,
            showarrow=False,
            font = dict(
                size = 14),
            bgcolor = 'white',
            opacity=0.85,
            text='Total Monthly Enplanements: <b>{}</b>'.format("{:,}").format(master_lst.at[i,'TRUE MONTHLY ENPLANED'])),
        go.layout.Annotation(
            x=.0,
            y=.9,
            showarrow=False,
            font = dict(
                size = 14),
            bgcolor = 'white',
            opacity=0.85,
            text='<b>{0}, {1}</b>'.format(master_lst.at[i,'MONTH_DESC'],master_lst.at[i,'YEAR']))
     ]


# fig.write_html("U:\moliver\In Progress\Airports\Animation Save\GPS_Enplanements.html")
# fig.show()

In [736]:
# LINE CHART

# Create Line Chart Figure
line = px.line(
    line_df,
    x=line_df.WEEK_STARTING,
    y='WEEKLY_ENPLANEMENTS2',
    width=1800,
    height=290,
    animation_frame=line_df.ORDER,
    animation_group=line_df.WEEK
)

# Formatting Updates
line.update_layout(
#     title = 'Estimated Traffic from Top 100 Airports Jan 2019-Dec 2020',
    xaxis = dict(
        tickmode = 'array',
        tickvals = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100],
        tickangle=-45,
        title = dict(text = 'Week (Starting on Date)'),
        ticktext = ['Jan 29, 2019','Mar 05, 2019','Apr 09, 2019','May 14, 2019',
                    'Jun 18, 2019','Jul 23, 2019','Aug 27, 2019','Oct 01, 2019',
                    'Nov 05, 2019','Dec 10, 2019','Jan 14, 2020','Feb 18, 2020',
                    'Mar 24, 2020','Apr 28, 2020','Jun 02, 2020','Jul 07, 2020',
                    'Aug 11, 2020','Sep 15, 2020','Oct 20, 2020','Nov 24, 2020'
                    ],
        gridcolor = '#e6e6e6'
    ),
    yaxis = dict(
        range = [0,20000000],
        title = dict(text = 'Weekly Enplanements'),
        gridcolor = '#e6e6e6'
    ),
    plot_bgcolor = '#ffffff',
    font_color = '#2e105e'
)


line.layout.updatemenus[0].buttons[0].args[1]['frame']['duration'] = 250
line.layout.updatemenus[0].buttons[0].args[1]['transition']['duration'] = 200
line.layout.updatemenus[0].buttons[0].visible = False
line.layout.updatemenus[0].buttons[1].visible = False
line.layout.sliders[0].visible = False


# for loop to update line color for each frame
for i, frame in enumerate(line.frames):
    line.frames[i].data[0].line.color = '#cc7639'




# line.write_html("U:\moliver\In Progress\Airports\Animation Save\Enplanement Line Graph.html")
# line.show()